# Data Read and Formatting

Notebook para la lectura de los datos y para su modificación de formato en archivos csv, con tablas individuales.

## Path Configuration

In [4]:
import os
from pathlib import Path
print("Directorio actual:", os.getcwd())

current_dir = Path.cwd()
project_root = current_dir
while not (project_root / '.git').exists() and project_root != project_root.parent:
    project_root = project_root.parent
    os.chdir("..")

print(f"Raíz del proyecto: {project_root}")
print("CWD cambiado a raíz del proyecto")


Directorio actual: /home/ferrus/Documents/university/semester_x/MA2003B/PROYECTO_MA2003B
Raíz del proyecto: /home/ferrus/Documents/university/semester_x/MA2003B/PROYECTO_MA2003B
CWD cambiado a raíz del proyecto


In [6]:
import pandas as pd

#Paths

path_2022_2023 = r"data/raw/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx"
sheet_names = ["SURESTE", "NORESTE"]
df = pd.read_excel(path_2022_2023,
                   sheet_name = sheet_names[0])